# Dilithium - PolyGlitches

In [1]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('polyglitches')

we append the path so that we can use the chipwhisperer library

In [2]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software']

In [3]:
import chipwhisperer as cw
import importlib
import json
import uuid
import numpy as np
from collections import defaultdict
dilithium = importlib.import_module("python-dilithium.dilithium")
dilithium.generic = importlib.import_module("python-dilithium.dilithium.generic")
logging.basicConfig(level=logging.NOTSET)
cw

<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>

In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'

NIST_SECURITY_LEVEL = 2

In [5]:
SS_VER="SS_VER_2_1"

In [6]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [7]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [8]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

In [9]:
#cw.program_target(scope, prog, fw_path)

In [10]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [11]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.adc.samples = 0 # maybe speedup?
    scope.adc.timeout = .002

def get_duration():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.loop(20)
    timed_out = scope.capture()
    assert not timed_out
    duration = scope.adc.trig_count


def get_poly_correct_packed() -> bytes:
    target.loop()
    poly = target.get_poly()
    return poly

def get_poly_correct() -> np.ndarray:
    poly_packed = get_correct_poly_packed()
    dilithium.generic._polyz_unpack(correct_poly_packed, nist_security_level=NIST_SECURITY_LEVEL)

duration = get_duration()
poly_correct = get_poly_correct()
dilithium2_polyz_unpack_settings()

294


array([-100330,   65070,    9596,   57850,   35652,  105533,  -12476,
        -46863,  -37129,  -56173,  -21068,    2324,  115540,   44633,
        125101,    3586,   69993,  -81687,  -64128,  120572,    2307,
        130526,    4041,  -41650,   84699,   -8418,   74822,  109619,
        -91410,  -91558,  115894,   23794,   38581,   97367,  -68344,
         13693,  -29260,  -93738,   25493,   28466,  -98993,  -35421,
       -129186,  -35972,  -98782,    5229,  121668,  -36138,   43042,
        126578,  -51112,  -26829,   88126,  -48827,   -6483,   91640,
         26814,  122947,   26573,   69248,   24721,  -59507,   95958,
         40139,  -14569,   50396, -112991,  -68730,   66400,  -78235,
        -86549,  -76860,   64682,  -31281,  -67048, -120548,  -94141,
        -16141,  -41256, -107494,  123510,   92927,  -46082, -106954,
        123712,  -54074,  -68055,  -27761,   95540,  -34279,   88614,
        -16752, -106508,  110918,  -79361, -111904,  -61989,  -37948,
        -98516,  -55

In [12]:
def is_perfect_glitch(faulted_poly: np.ndarray, correct_poly: np.ndarray = correct_poly, coeffs_per_iter: int = 4) -> bool:
    # zero good iters would be total and 64 good iters would be no glitch
    for i in range(1, 63 + 1):
        # index of the first zero coefficient due to a fault
        split_index = i * coeffs_per_iter
        
        same_as_expected = np.all(faulted_poly[:split_index] == correct_poly[:split_index])
        rest_faulted = np.all(faulted_poly[split_index:] == 0)
        
        if same_as_expected and rest_faulted:
            return i
    return None

In [13]:
glitch_sig_no_1 = b'\xf3k\xa5.\xeb\xdb\xf2\xc1v\x94?6\x95\xfc\x1d\xefk\xc6p3 \xda+7\xa4\x1a\xe5"k\xbb\xbf\x17\xec\x87\x97\x07d\xa7\x1d\x85G\xb8tI\x8f\x01\x0b\xa3\xc3\xad\x02\x91\x82m\xab$\xe5\xba}\xa0<\xe0F\x95tA\x81|\xa6\xeeD\xfcl\xa7o>\n\x02\xf7u\x08p\x03_\xaa\xa80\xb5\xb8\x83H\xbcM\xf3\x14\ne\xa7\x96\x1d\xb5\xc3\xc3hJ\x13\x91i\x9a\xb00\x9fr>r\xda\xb8\xcd\xc5\x194d\xac\x82\x83):\x8a?"\xa3\xe2\x81C\xae\xa7K\x82K\xa3\xdaWUF0z\xec0\x9a\xd1\xa7$\xfb\x1a\x96\xa1~&F\x97\xcd~0\x83\xd9a<u\x9f\xd9\xa1{\x92H\xd1X\xe08\x96\xecT\x95\xbb\x1d\xc3\x98\xfct\xc6\xcc u\x0f\xcbN\x03\x05\xe9i^0\xb7\xf5\xbao\x1f\xfc\xc8\x12\xaa\xf7\xe8\x94\x1d\xecS\xa2A\xebp\xe8\xd0\x1c\xe0L\xdb\x9d\xf0\x18\x9b\xca\x8ah\x17\x9a^J\\\x90\x11\xa0\xf3:\xb1_3H\xed\x0b\xf2\x0eQ\x9a\x0c\xb87\xb6\xd4f\xbd\x7fj\xf4\'\x875\xe87\xf1\x8e\xaa\x05\xd0\xd7\x8f+&uAb\xbdS\xbb\n\xbc\xf6\xf7\xc2-\x03\xaf\x1c\x93\x89[+\xb0|O\xdf\xee&\x1a\xdcB@\xdf\xa3\xf7Qq\xe3\x0b|^dKdC\xa3\xf4\xbc\xe5\x19\x85\x9c\x8a\xbd\x99,\xcf:\xb4\xbc\x89)]X\x97\xd4q\xbey\x9e\x8d\xd3\xa1p\x17B\x1e\xf4K/\x0f\x01\xec\xbe8F\xe8\xee\x1c\xb4\x98H\x03\xcf\x80\xbf\x93\x91\xf4\xe9\xaeN\xf15\xe8{\xa6\xcd4\\[\xf0\x06\xa0\xbb\xcb\x9f\x8f\x88\xf68\n\x1ftSi\x15\xa5\xc7\xf9\xad\xe2\xe9\x87\xe2e\'\x9d\xce(\xab\x88\xdc6\xa8\t!\xbd\xea\x82\xf8\xa6\xbf\xf32R\xbe\xc21~\xbb\x19s \x8d\x81c\xa2\xe7\xe2\xc7\'\xba\xddaI\x96\x97\xa0G\xe2\x84\x1d\x91\x02\xadw\x15\xa5\xf6\xcd\'\x91\xaf\xcc~5\x10H\'\xbc\xfc4u\xb0\x03\xd7.\x01\xb0\x91_Q-\x80\x10x\xa1\xa1\xfd\xf5\xf5\xc5\xc0#\xc1\x13\x13X\xc0\xbd\x95\xbfam\xea=\x85\x9c\xfa:\xe9\xfa4*\xa9\xe0\xc8,\xd8\xf62^\xb3\xd6\xf9\xfe\xf6\x1b[A\xea(\xa7\xa5\xf1l{)`!\xb4\x1b\xfc\xben\xe2\x01\x16\xbb\x95\xa2GVd\x0f\xc2\x84A$\xa1\x0e\x84-\xd7\xa83\x05\xff\xa2\x04!\x0c\x1bK\xad\xb2\x11(\xa7\xcf\xd5m\xcb>\x9aYU\xa6\xf59\xb9aI\xa1\x066\x0b\xb3!\xd6`\x87n\xb9\xa37\xfb\x9c?\xd8;H\xa9\xac\xc5t\xd2\xde\xf9-\xab\xa9\x14\xdf3\xd2\xa8\x11\xe8\xde\x89>\x82\xe1\x81\x1a\xa0\t\xad\xa7\x0bn\xab\xad\xdd\xb4\x88\'\x9c\x103\xf3\xa9\x17\xf0\xb2\xbf\x04\xc4\xbf\x98\xd6/_&\xc9\xf7\xbc\x89dVa!\xca\xde\x08^p\xcfl<%@\xf5o\x86V\x15\x8c\x9aH\xc4\x98\x0fMk\xad<>\x13V\\\x9b\xd2\xe7A+\xf7\'v\xad\xbe\x9e\xa9\xe5\xe2\xec\xa8\x85\xcf\x04\xae\x94?!#\x9c*\x19\x1d\xc5k\xcb\xcf\xe1*\x83\'\xa8\x93\x8b\x1e\xd7}\x90\x16\xb8a\n\xed\x0b\xb7I\x06\xdf\x8bj\xe8\xd9\xd8\xac\xc4w9\\\xeb\x1f\x9d;2\xdc\xf5^\x84\xc1\n\x1f\'\x11\xfdo\x82\xa7G\\\nv\xd4\x08\x8f\x02\x93hq\xeb\xc3\xbe\xaf\xe9K\xab\xb8`@>2\xd3y\x16"9\xff\xc9\x05\xf9\x12\x8b\xc2\x032\xc9\xf8D\xf5d\xa5\x06\xe0i\xfaZ\xcbQ\xc6Y\xb6G`\x0ee\xad\x93\xc4\x8fhW\xa2x\xe1b\x85\x82Z9\x11\xbff\x92\x1e\xca\xb9K\xb3\xe2$\x01\xe7\x08"\x86\xe925Nx\xb0,\xea\x08R\x92\x08xL\xa2\x8a\xd3\x8d\x98$b\x92\x807bJ\x14\xd5\xf1@\xf3\xee\xc0\x928\x05\xb8\x16V\x89P\xfa\x17\x96_%\xac04\xb2\xfa6\xcdP\ty]\x94\x95\xd4\xcb&m\x00\xfbf\xd7n\xca4x3g\x89\x06\x1ej;\x10$\x93vYt\x1eS\xf8cH\xc0\x15-\xd3\xd7\x8a\xee(\xd8r\x0eF-4\x93\xcd\xf1UOb^\x0f}\x15;\xcd\xbd1\x17H\x02m\xb3B\xb4\x1bOy?N\x93@&\x9a\xdf\xf3cBcr\r\xaee\xdd;\xc6k\xe3J4c-/0\x86,\xa9\x8cG\\\x03f\xc1\x8a_\x0bfS\xe7v\xbd\xf3\x10\xe0I\x8c\xc1\xc1\'\xa9\t\'e\xd8\xf3\x1d\xd5V\xce\xf66)6\xbc=\xd8\xc2oY2\xf4%\xb0\x97{~\x02@\xf5\x04&8V\xd3\xfds\xf6b\xc0\xa4\xc1\x12\xb5>\xb2\xdbY\xf5\x83\x84\x9bm2Y\x81D\x9e\xd0\xcc M\xdbq\xafA(\x92GG\xf6\x17\xc4\xa6\x0e\xa4\r\x83O9\x15\xb1\xf7"\xa5G\xdd\xbbx%4\xd9\xd2E\x18\xc9\x16\xc6Ax\x9f\x14\xcc\xd0\xb0\x81\x90\x96u\xae\x07\x9f\'")\x8b]\xd9\xfa*\xbd\xd4\xach3\x88\xf0\x0fk\xfeh\xd4\xa4,\xdd\x1f\x96\xb9!\x7f/\xa5\x10\x9f\x9d\xb0Z\xec\xbe\x91\xbc\x88#\x04\xd4\x08\x8eE\xe0\x90\xfc\xa9\xbdu\x84\xbaT\xb8f\x19\xc6|\xb7:\x9e\x96gg\xe2\x18/Q\x198\x1a\xa3\xd2\x89\xf7\x19\x19b\xa9$\x18\xfd\x8aa8\xdc\xab\xab@\xc0t\xc3:QI\x1a\\O\xd554\xcda\xd2~\xf1\x0bbI\xa3\x0e\xa5"2\xce\x8d\xee\x9b\x97\xf0\x92\xf5Z^\x12\xe4\xde\x10V\xdf\x9cYQ\xcbP\xe4\xf3*\xc2\xb4^%zV\x03Z,\xe3\xf4\x9e\x1c<\xab\xb2w(\xc0.\x15l\xac\xb5k\xcd\x9a\xf1*\x14\x04M\xdeoq)0;\xebV\xaa=\xb78c\x80\xc4|!|\xc6\xf9\xf2T\x04\x18\xa9f\xf5\xfc\x12\x04\xf4\xce\xaaA\xab*9):`E\xc8\xe0\xa4\xe7\xe1W7\xbcfb\xbbo\x7f\xa9\xeb\xb8\xd5\xa6\xff{(\xdc\x1d\xfdL[\'\t\xa6\x17\x0f\xd4\x80\x02\x9c}0\x0b\x9ai\x04\x19\xbf\x0c\xce\x85\xfc\xab\x1am\xe5\xc6`<\x1d\x8d\xfc\xd6\x1f\xa0\x84\xa1\xc7\xe3\x06m\x98~\xd6I0\xa7\xf7(Z@"\x1f\xfe@\xb0@\xdc\t\x16D\xffTT\xfa\x8a\'\x01N\xb8\xaf\xd1\xd5\xff\x1cr\xec\xc9\r\xba\xef#\xd9\xec\xd8u_\xce\xfcC \x7f0w\xca>\xfa"\xd4IR\xfd\x9c$B\xf7\x00_d\xf4\xe2$\xbdP\x94\x1f\\lQ\xb5\xd0 \x84\xadt\x95y\x18\xa9L~\xc0>\xb5\xab\x9fbM\x92FC\'U\x90\x05E\xeb\x90\xf2\xc8\x8bw\x9b\xb6\xe3\x9e\xd5R\x87\x02?]\xba\xab.\x0b\x97,W\x96\x8b>\xb4\xcb[1P\xb7K\xb7\xbc\xde\xec\x9b*\xed\xcd\xc8\x9d\xd6@\x86u\xfe\x0c\xa0\x11\x1b\xb3\x95\xba\xe9\x93\xeb\xc4BKQ\xff\xf4\xd9+|\xc6\x97v\x7f\xd5\x1b\x0e\xb15\x931(j\xab\x80\x1a\xcb6\x834\x86;\xc7\x16@\xe2%\xdfhu\xce\x8f\xf2e\xd02\xa1\nU\xa0\n0\x9f\xd5(W\xcar\x08\xfc+M[^\xcd\x9b\x8f\x14\x8a\xbd\x9c\xe3\x01\x84\x00\xac\x19\xfd\x1b\xf1\xdd\xa6\x8a\x9aG\xecj\xe1(#\x83\x8b\xa3\xdb\xa88\xfe\x85w\xb3@)\xc0\xa0\'\x82\x15aeX5V\x86P\xabU\xb0\x8eIp\xfb\x88\\N\xe5\x1dl\x9f\xf8\xe7"\xda\xca\x85\x8c(i\x04c(\xcf\x91bb\xc6\x1f\xe9\x1c\xf1\xday\xf9\xef99$\xae\xe6\x84y\xe1N\x17o\x1bRp\x84<#G_\xb0M:DY,H\xa8/\x12\x9e\xf6\x037\x08\xe0t\xaa\r\x1359\x199\x94\x13n|K\xe5\xf44\xd0\xc9seb\x1ft\xd1\x05 >\x9a\xbcC3,\xda\xdc\xce\xdc\x8e\xb8\xc8Dx\x18\xae\x01\x05\xb4\x96d?\xc7n\xf6\x17\x04\xb2\xa3\xee\x86\xca\x86\xdb\xedh\xfd9\x12\xed\xa9\xe3\xe8\x85\x036\x142/\xd0\x98fZ\xddqg\xfe\xed+o\xfe\xfd\xd9\xd2\xf9]\xafG\xf4\'\x9a\xfa\x10\xd9\xf5\xceXE\xa2?\x11\xc5\xf8\xdc%\xdf\x84\x18\x88\xf9\xab\x12\xc2,\xa7\xe1\xa7\xce\xa0\xfe\x99^D\xabi\xaf\xbf\x0fQ\x95\x02\tz"\x83kd\xd4M\xb2]\x93\xb19\x90kf2\x93\x17\xb0\x8a\xba\xbf\xa1\xc4}[\x91\xb8\xda8\x1b\x81\x0f\xef\x8b\xaaa\xfb\xdb],1IX\xc1[(\xca3\x837\xd9\xb6\x97Z\x97\xe7#i\x92\x1f\x88\x06\x07eT\x1d$C;\xb8h\xb1\x92\xce]\'7\xb0\xd4\x87\xbf\xb0\xe1\xaao\x18\xe5_\r\x8a\xd9\xa5"\xa8\xd9\x07\xa9\x9a\x8c\xf7=\x9aK\x0e1\n\xb8\xd7\xaf \n\\~\'\xdb\\`\xd3\x1b\x07\xb2\xea@\xc1\xba&\xa7\xbd\\H?%\x1e\x83(\x12*\xd5\xc0\x1a\xe3w\xc3Ik\xb3\xd2M~\xa1V\x15\x01H&\xbf;b\x16\xb5]\xc1\xf6\xc9\xdc\xe7\x1e=\x0b!&&\xb6_8M\x9b\xa93j\x83b\x9c\x06\x1f\xb0\xdbu);o\xd7}\x0f\xb9T\x89`\xe8\xec\xcb\x8c!6\xff\xda\\\x87\xd5Uns\x08\xa8)aqIo\xf1\xb8\x1b2ZZ\'N\x9cr\x0fbq\xa3\xb56LZ\x839F\xe3\xc8\xd2I\xdf\xd3-\xd7n\xf1\xd6\x16%\xf9k\x8fQ\x90\xc8Qd*\x0c\x008\x96^;\x9d\xd1\xce\xc5\xee\xd9\xb5R\xec\x1452R5\x8a\x12\x82Y<K\x80z\x88\x9e&\x8b\xf4l^\x99\x1f:\xa5\xc52\xda\xad\xea}\xd1\x95g9\xb0\xc1\x1f\xccL\x94 CQ\xcaVm|\xbdd\xb1\xaf\x1aY\xb6(\x07\xf9\x94\x96\xaa\x86X^ji\x13\xc5\x04\xc1\x18\xbc\x8c\x83\xe6\xf5 \xe9^h\xf7\xbf\xc5\x80\xa2\x1bI\xd2\xec+\xa5\xaa\xfc0\x8fsy\x15\xdc\xc7f\x81\xf2\x99\x86\x14\xb8\xe2\xed\xb7\xf5&\\\xac~t\x1c\x04b\xd28?\x1d@e\xbd\x10\x18\x1b-;ARaen\x83\xa3\xa9\xaf\xc3\xdc\x00\t\n\x10\x12\x16"\'>Zemvxz\x80\xe9#knrz\xb8\xbb\xd2\xf8\x0b\x160Jalrt\x85\x8b\xa9\xb2\xbf\xe0\xe9\xf1\xf2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10!*;'

In [14]:
glitch_spots = [i for i in range(2000, 3000)]
width_start = 7.03125
width_stop = 7.03125
offset_start = 7.03125
offset_stop = 7.03125
repeat = 1
step = 1
redo = 1

In [15]:
print(glitch_spots)

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165, 216

In [16]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

In [17]:
faulted_polys = []
faulted_poly_params = []
perfect_db = defaultdict(lambda: [])

In [18]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from chipwhisperer.capture.targets import TargetTimeoutError, TargetIOError
from tqdm.notebook import tqdm
import re
import struct
import numpy as np

scope.glitch.repeat = repeat
gc.set_range("width", width_start, width_stop)
gc.set_range("offset", offset_start, offset_stop)
gc.set_global_step(step)
scope.glitch.repeat = repeat
target.reboot_flush()

b'\x00\x0bb\x07boot ok\xc1\x00'

In [21]:
def one_try(scope, target, gc, is_faulted, faulted_polys, faulted_polys_params, perfect_db) -> bool:
    scope.arm()
    try:
        target.loop_send()
        
        trigger_timeout = scope.capture()
        if trigger_timeout:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            __LOGGER.info('fail due to trigger timeout')
            target.reboot_flush()
            return
        
        target.loop_recv(timeout=10)
    except TargetTimeoutError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info('failed due to read timeout: {e}')
        target.reboot_flush()
        return
    except TargetIOError as e:
        gc.add("reset", (scope.glitch.width, scope.glitch.offset))
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return
    else:
        candidate_poly_packed = target.get_poly()
        candidate_poly = dilithium.generic._polyz_unpack(candidate_poly_packed, nist_security_level=2)
        if is_faulted(candidate_poly):
            perfect_i = is_perfect_glitch(candidate_poly)
            if perfect_i is not None:
                __LOGGER.info(f'PERFECT after iter {perfect_i}')
                perfect_db[perfect_i].append(scope.glitch.ext_offset)
            __LOGGER.info("Glitched! Ext offset: {} Offset: {} tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success", (scope.glitch.width, scope.glitch.offset))
            faulted_poly = candidate_poly
            faulted_poly_param = (scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width)
            faulted_polys.append(faulted_poly)
            faulted_poly_params.append(faulted_poly_param)
        else:
            gc.add("normal", (scope.glitch.width, scope.glitch.offset))

In [22]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from chipwhisperer.capture.targets import TargetTimeoutError, TargetIOError
from tqdm.notebook import tqdm
import re
import struct
import numpy as np


gc.set_range("width", width_start, width_stop)
gc.set_range("offset", offset_start, offset_stop)
gc.set_global_step(step)
scope.glitch.repeat = repeat
target.reboot_flush()

is_faulted = lambda candidate_poly: not np.all(candidate_poly == correct_poly)
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in glitch_spots:
        scope.glitch.ext_offset = i
            for j in range(redo):
                one_try(scope, target, gc, is_faulted, faulted_polys, faulted_poly_params, perfect_db)

(ChipWhisperer Target WARNING|File SimpleSerial2.py:508) Unexpected frame byte in CWbytearray(b'84 72 80 cd 02 6f 20 93 78 5b 37 b0 8c 4f 9f ee 26 11 dc 5a 40 3f a4 f7 4d 71 e4 0b 64 5e a4 4b a4 42 a3 02 bd 11 1a 25 9d 0a bc 99 28 cf 2e b4 cc 89 a9 5f 58 92 d4 69 be d9 9e 4d d7 a1 65 17 32 1e 04 4b 6f 10 01 f0 be 28 46 f8 ee 5c af 98 03 cb 80 cf 93 91 f8 e9 b6 4e f1 35 e8 7c e6 cb 34 55 5b 14 07 f0 bb 0b 9e 8f 8e f6 6c 0a 0f 74 53 68 15 9c c7 05 ae e2 e9 c7 e3 65 1d 9d ca 28 eb 88 f6 00 03')
INFO:SimpleSerial2Dilithium:got an BlockingIOError exception: target logger logged a warning during simpleserial_read: ["Unexpected frame byte in CWbytearray(b'84 72 80 cd 02 6f 20 93 78 5b 37 b0 8c 4f 9f ee 26 11 dc 5a 40 3f a4 f7 4d 71 e4 0b 64 5e a4 4b a4 42 a3 02 bd 11 1a 25 9d 0a bc 99 28 cf 2e b4 cc 89 a9 5f 58 92 d4 69 be d9 9e 4d d7 a1 65 17 32 1e 04 4b 6f 10 01 f0 be 28 46 f8 ee 5c af 98 03 cb 80 cf 93 91 f8 e9 b6 4e f1 35 e8 7c e6 cb 34 55 5b 14 07 f0 bb 0b 9e 8f 8e f6 6c 0a 0f 74 53 68 1

INFO:polyglitches:Glitched! Ext offset: 2159 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2160 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2161 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2162 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2163 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2164 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2165 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2166 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2167 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2168 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2169 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2170 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2171 Offset: 7.03125 tWidth: 7.03125

INFO:polyglitches:Glitched! Ext offset: 2309 Offset: 7.03125 tWidth: 7.03125
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a
INFO:polyglitches:fail due to trigger timeout
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:polyglitches:failed due to read timeout: {e}
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2325 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2326 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2327 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2328 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2329 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches

INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2392 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2393 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2394 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2395 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2396 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2397 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2398 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2399 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitch

INFO:polyglitches:Glitched! Ext offset: 2462 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2463 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 32
INFO:polyglitches:Glitched! Ext offset: 2464 Offset: 7.03125 tWidth: 7.03125
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:polyglitches:failed due to read timeout: {e}
INFO:polyglitches:PERFECT after iter 34
INFO:polyglitches:Glitched! Ext offset: 2471 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 34
INFO:polyglitches:Glitched! Ext offset: 2472 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 34
INFO:polyglitches:Glitched! Ext offset: 2473 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 34
INFO:polyglitches:Glitched! Ext offset: 2474 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 34
INFO:polyglitches:Glitched! Ext offset: 2475 Offset: 7.03

INFO:polyglitches:Glitched! Ext offset: 2537 Offset: 7.03125 tWidth: 7.03125
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 
INFO:polyglitches:failed due to read timeout: {e}
INFO:polyglitches:PERFECT after iter 35
INFO:polyglitches:Glitched! Ext offset: 2544 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 35
INFO:polyglitches:Glitched! Ext offset: 2545 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 35
INFO:polyglitches:Glitched! Ext offset: 2546 Offset: 7.03125 tWidth: 7.03125
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0a
INFO:polyglitches:fail due to trigger timeout
INFO:polyglitches:Glitched! Ext offset: 2562 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2563 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2564 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Gl

INFO:polyglitches:Glitched! Ext offset: 2712 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2713 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2714 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2715 Offset: 7.03125 tWidth: 7.03125
(ChipWhisperer Target WARNING|File SimpleSerial2.py:508) Unexpected frame byte in CWbytearray(b'84 72 80 f5 7e 30 83 19 60 3c 6f 9f cd a1 ab 92 48 cd 58 e9 38 92 ec f4 95 bb 1e c3 a0 fc 6c c6 5c 21 35 0f cb 56 03 c5 e8 89 5e 30 b9 f5 bd 6f 37 fc 88 12 aa f9 e8 8a 1d 04 54 22 40 ab 72 e8 c0 1c e8 7b 9d 70 1c 9b cc 8a 9c 17 aa 5d 0a 5c 90 0c a0 eb 3a 11 60 33 48 ed 25 f2 f2 50 4a 0d 78 31 b6 de 66 99 7f 6a f5 e7 83 35 e0 37 e9 8e ea 05 50 d5 8f 27 26 7d 41 e2 bc 13 b9 0a b1 f6 23 c3 47 00 03')
INFO:SimpleSerial2Dilithium:got an BlockingIOError exception: target logger logged a warning during simpleserial_read: ["Unexpected frame byte in CWbytearray(b'84 72 80 f5 7e 30 83 19 60 3c 6f 9f

INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2802 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2803 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2804 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2805 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2806 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2807 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2808 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitched! Ext offset: 2809 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:PERFECT after iter 38
INFO:polyglitches:Glitch

INFO:polyglitches:Glitched! Ext offset: 2956 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2957 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2958 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2959 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2960 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2961 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2962 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2963 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2964 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2965 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2966 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2967 Offset: 7.03125 tWidth: 7.03125
INFO:polyglitches:Glitched! Ext offset: 2968 Offset: 7.03125 tWidth: 7.03125

defaultdict(<function __main__.<lambda>()>,
            {28: [],
             32: [2325,
              2326,
              2327,
              2328,
              2329,
              2330,
              2331,
              2332,
              2333,
              2334,
              2335,
              2336,
              2337,
              2338,
              2339,
              2340,
              2341,
              2342,
              2343,
              2344,
              2345,
              2346,
              2347,
              2348,
              2349,
              2350,
              2351,
              2352,
              2353,
              2354,
              2355,
              2356,
              2357,
              2358,
              2359,
              2360,
              2361,
              2362,
              2363,
              2364,
              2365,
              2366,
              2367,
              2368,
              2369,
              2370,
           

In [25]:
result_data = {
    "glitch_spots": glitch_spots,
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
    "step": step,
    "faulted_polys": list(map(lambda np_poly: np_poly.tolist(), faulted_polys)),
    "faulted_polys_params_descropton": "0 = ext_offset, 1 = offset, 2 = width",
    "faulted_polys_params": faulted_poly_params
}
if not test_one:
    with open(f'77param{uuid.uuid4()}.json', 'w') as f:
        json.dump(result_data, f)
json.dumps(result_data)

In [24]:
scope.dis()
target.dis()